# 1. Importações

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 2. Leitura dos Dados

In [5]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/03. Dados para treino e teste 100dim/'
x_train = pd.read_csv(path + 'x_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
x_test = pd.read_csv(path + 'x_test.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_train = pd.read_csv(path + 'y_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_test = pd.read_csv(path + 'y_test.csv', index_col=False).drop('Unnamed: 0', axis=1)

len(x_train), len(x_test), len(y_train), len(y_test)

(1106, 277, 1106, 277)

In [6]:
x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,1.380696,1.117377,0.137957,0.761084,0.534715,0.552442,-0.004785,0.152474,-1.161453,-0.055611,0.132354,-0.807359,-0.048969,0.822576,-0.392735,0.761977,-1.249718,-0.981962,-1.144687,0.978987,-0.320123,-0.708886,1.545143,0.277308,-0.886935,-0.091715,0.015212,1.188224,-0.579227,0.109587,0.628864,-0.653652,-1.361550,0.359014,0.358086,0.527734,0.379979,-0.916307,0.628407,0.948281,...,-0.139626,-1.028920,-0.002837,-1.452423,-0.445462,0.631283,-0.455308,-0.178009,-0.008535,-0.305238,0.184161,0.504119,0.575579,-0.408739,0.025195,-0.258042,-0.553967,-0.371115,-1.129306,-1.563835,-1.393074,0.807052,0.491048,0.359533,0.769408,-0.611857,-0.204234,0.872937,-0.285450,0.107422,0.427236,-0.796847,0.424916,0.063506,0.334243,-0.162930,0.768123,0.044175,-0.354515,0.294467
1,1.256561,-0.074913,0.145955,1.136830,0.004156,-0.870302,-1.605373,1.210091,0.282945,0.002317,-0.053097,0.487903,-0.638106,1.438257,0.339600,-0.661097,-0.211923,-1.258146,-0.357708,1.697232,-0.369264,0.859936,-0.557076,1.936877,-1.336052,-0.530775,0.261859,-0.710866,-0.785438,1.886991,-0.154471,-0.065372,-1.835043,-0.575359,-1.162260,0.632790,0.197085,-1.412120,-1.672758,0.127306,...,-1.585380,-0.303114,1.383609,-2.145113,0.153745,0.732192,-0.416609,0.694549,-1.263390,-1.328003,-0.351537,0.556735,0.410066,-0.833729,-1.081671,0.547176,0.297076,1.510936,-0.639551,0.223967,-1.502099,0.028978,1.489418,-0.645874,1.020926,-0.072282,-0.765184,1.740980,0.399647,1.177060,1.146743,-0.143019,-0.592275,0.039823,0.014546,0.103534,0.103899,0.478186,1.674268,1.021356
2,1.730329,0.072701,-0.001182,0.384270,0.467653,-0.644043,-0.718277,0.759209,1.150164,0.657574,-0.747905,0.727604,-0.094454,-0.702027,-0.745317,-0.037272,-1.346404,-0.873271,-2.331137,0.419611,-0.827884,1.455137,0.439708,2.281665,-1.272789,0.117314,1.122796,0.140376,0.022633,1.656343,0.727345,-0.550547,0.857728,-3.204981,-1.283571,2.088672,1.807675,-0.619398,-0.214990,0.144727,...,-0.642876,-1.383184,0.484584,-1.547997,0.054054,0.037525,0.964339,1.605726,-0.496296,-0.093266,0.642300,1.501297,0.197259,-1.480613,-0.577922,-0.850485,-0.065439,-1.388217,-2.119327,-1.221099,-0.591608,0.925732,0.328761,-0.430016,0.720029,-1.482438,-0.850712,1.561310,0.397910,0.820241,0.095608,0.235877,-0.841214,-0.966722,-0.783702,0.168322,0.311457,-0.297746,1.023306,-0.754920
3,-0.001383,-0.068355,-0.176925,0.785359,0.598387,0.482275,0.116769,1.513961,-0.517657,0.281069,-0.977379,1.359786,-0.103350,0.244982,-0.066079,-1.395389,-0.845733,-0.091960,0.151931,1.528891,0.338386,0.797338,0.254699,1.203377,-0.300761,-0.227034,0.784468,0.145645,0.430758,0.689033,1.378608,0.754982,0.689577,-0.696048,-1.606491,0.807161,0.621024,-2.588973,0.345826,0.778142,...,-0.893017,-1.001482,-0.333357,-0.619940,-0.722008,-0.088832,-0.160842,0.865481,-0.286700,-0.892267,1.494818,-0.566854,0.374465,-0.466815,-1.239538,-1.934977,0.195195,-1.389119,-1.112421,-0.776210,-1.135332,0.501908,1.364557,-0.345560,-0.202960,0.310760,-0.801793,0.208756,-0.053151,-0.710049,0.462919,-0.398455,-0.039614,0.865347,-0.418910,0.023014,0.069128,-0.570670,0.556745,0.301391
4,0.708433,-1.674911,-0.414836,0.604649,2.959635,0.938199,-0.836568,-0.111802,-0.676853,-0.356687,-0.307003,0.156009,-0.103471,1.101890,0.016957,-0.407898,-1.596707,0.408511,-0.668082,1.289135,-0.618351,0.795663,-0.317458,2.704531,-1.645182,-0.139670,0.375674,-0.407643,-0.265826,1.797180,1.028588,0.391431,0.765697,-2.112264,-0.527269,1.146115,1.394309,-1.534569,0.219000,0.609186,...,-0.740411,-0.015810,1.007980,-1.554050,-0.253225,-0.373098,1.658329,-0.466901,0.022074,0.167850,-0.929536,2.192057,-0.476363,-2.080670,-1.394679,-1.471324,-0.659242,-1.017550,-2.090855,-0.536676,-2.221824,-1.095451,1.896417,-0.168267,-0.835030,-1.185452,-1.194960,1.090425,0.132347,0.394848,0.448447,-1.555806,-1.08752

# 3. Regressão Logística

In [7]:
class LosgisticRegression():
    def __init__(self, alpha=0.001, epochs=100):
        self.alpha = alpha 
        self.epochs = epochs
        self.errors = []
        

    def sigmoide(self, x, w):
        return 1/(1+np.exp(-x @ w))

    def loss_cross_entropy(self, sum_error):
        return (1/(2*self.N))*sum_error

    # def gradient_descendent(self):
    #     return True
    
    # def stocatic_gradiend_descendent(self):
    #     return True

    def fit(self, x_train, y_train):
        self.D = x_train.shape[1]
        self.N = x_train.shape[0]
        self.w = np.zeros(self.D + 1)
        x_train = np.hstack((np.ones((self.N, 1)), x_train))

        for epoch in range(self.epochs):
            sum_error, sum_ = 0, 0
            for i in range(self.N):
                current_error = y_train[i] - self.sigmoide(x_train[i], self.w)
                sum_error += current_error**2
                sum_ += current_error * np.transpose(x_train[i])
            self.w = np.transpose(self.w) + (self.alpha * (1/self.N) * sum_)
            self.errors.append(self.loss_cross_entropy(sum_error))

    def predict(self, x_test):
        x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
        return [1 if self.sigmoide(row, self.w) >= 0.5 else 0 for row in x_test]
        

## Treino

In [8]:
num_epochs = 250
alphas = [0.01, 0.001, 0.0001, 0.00001, 5, 3, 10]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, y_train["classification"], test_size=0.2, random_state=42)
len(X_train), len(X_validation), len(Y_train), len(Y_validation)

(884, 222, 884, 222)

In [10]:
print("------ GRID SEARCH ---------")
accs = []
for alpha in alphas:
    lg = LosgisticRegression(alpha, num_epochs)
    lg.fit(X_train, Y_train.values)
    y_predict = lg.predict(X_validation)
    print(" ------------ ALPHA = " + str(alpha) + " --------------------")
    print('Test Accuraccy: ', accuracy_score(Y_validation, y_predict))
    print('\nConfusion Matrix:')
    print(confusion_matrix(Y_validation, y_predict))
    print(classification_report(Y_validation, y_predict, output_dict=True))
    accs.append((alpha, accuracy_score(Y_validation, y_predict)))
    report = classification_report(Y_validation, y_predict, output_dict=True)



------ GRID SEARCH ---------
 ------------ ALPHA = 0.01 --------------------
Test Accuraccy:  0.7702702702702703

Confusion Matrix:
[[100  11]
 [ 40  71]]
{'0': {'precision': 0.7142857142857143, 'recall': 0.9009009009009009, 'f1-score': 0.796812749003984, 'support': 111}, '1': {'precision': 0.8658536585365854, 'recall': 0.6396396396396397, 'f1-score': 0.7357512953367876, 'support': 111}, 'accuracy': 0.7702702702702703, 'macro avg': {'precision': 0.7900696864111498, 'recall': 0.7702702702702703, 'f1-score': 0.7662820221703859, 'support': 222}, 'weighted avg': {'precision': 0.7900696864111498, 'recall': 0.7702702702702703, 'f1-score': 0.7662820221703859, 'support': 222}}
 ------------ ALPHA = 0.001 --------------------
Test Accuraccy:  0.5990990990990991

Confusion Matrix:
[[109   2]
 [ 87  24]]
{'0': {'precision': 0.5561224489795918, 'recall': 0.9819819819819819, 'f1-score': 0.7100977198697068, 'support': 111}, '1': {'precision': 0.9230769230769231, 'recall': 0.21621621621621623, 'f1-sc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


 ------------ ALPHA = 10 --------------------
Test Accuraccy:  0.7927927927927928

Confusion Matrix:
[[96 15]
 [31 80]]
{'0': {'precision': 0.7559055118110236, 'recall': 0.8648648648648649, 'f1-score': 0.8067226890756303, 'support': 111}, '1': {'precision': 0.8421052631578947, 'recall': 0.7207207207207207, 'f1-score': 0.7766990291262136, 'support': 111}, 'accuracy': 0.7927927927927928, 'macro avg': {'precision': 0.7990053874844592, 'recall': 0.7927927927927928, 'f1-score': 0.7917108591009219, 'support': 222}, 'weighted avg': {'precision': 0.7990053874844593, 'recall': 0.7927927927927928, 'f1-score': 0.791710859100922, 'support': 222}}


In [11]:
lista_ = [acc for alpha, acc in accs]
best_alpha = accs[lista_.index(max(lista_))][0]
best_alpha

10

In [12]:
lg = LosgisticRegression(best_alpha, num_epochs)
lg.fit(x_train, y_train['classification'].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


## Predição

In [13]:
y_predict = lg.predict(x_test)

## Métricas

In [14]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.7256317689530686

Confusion Matrix:
[[126   4]
 [ 72  75]]


In [15]:
report

{'0': {'f1-score': 0.7682926829268293,
  'precision': 0.6363636363636364,
  'recall': 0.9692307692307692,
  'support': 130},
 '1': {'f1-score': 0.663716814159292,
  'precision': 0.9493670886075949,
  'recall': 0.5102040816326531,
  'support': 147},
 'accuracy': 0.7256317689530686,
 'macro avg': {'f1-score': 0.7160047485430606,
  'precision': 0.7928653624856157,
  'recall': 0.7397174254317112,
  'support': 277},
 'weighted avg': {'f1-score': 0.7127957417397246,
  'precision': 0.8024701615616937,
  'recall': 0.7256317689530686,
  'support': 277}}

## Salvando resultados

In [16]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str({'alpha': best_alpha})
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica_100dim.json')

In [17]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica_100dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.80247,0.725632,0.712796,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...",{'alpha': 10}


# 4. Análise Discriminante Gaussiano

In [18]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminant():
  def __init__(self):
    pass

  def fitAGD(self, X, y):
    self.__classes = np.unique(y) # pegando as classes e ocorrencias
    numClasses = len(self.__classes) #numero de classes
    n, numFeatures = X.shape # numero de linhas e colunas do dataset 
    
    self.__sigma = 0
    self.__probabilidadeClasses = np.zeros((numClasses, 1))
    self.__media = np.zeros((numClasses, numFeatures)) # criando a lista da media das classes por feature

    for i in range(numClasses):
        # flatnonzero retorna os indices em que os valores são diferentes de zero. O retorno é uma lista flat. 
        # A comparação y == classes vai retornar um array de 0's e 1's. 
        # E portanto a operaçao final vai pegar todos os indices em que a classe é igual ao y.
        indices = np.flatnonzero(y == self.__classes[i])

        self.__probabilidadeClasses[i] = len(indices)/ n
        self.__media[i] = np.mean(X[indices], axis=0)
        self.__sigma += np.cov(X[indices].T) *(len(indices)-1)
        
    self.__sigma /= n
    
    return {'media': self.__media, 'covar': self.__sigma, 'classes': self.__classes, 'numRows': n, 'numClasses': numClasses, 
            'numFeatures': numFeatures, 'probabilidadeClasses': self.__probabilidadeClasses }

  def predict(self,X):
    # Usando a distribuição gaussiana multivariada da biblioteca scipy.stats
    # Basicamente o calculo da função densidade de probabilidade visto na imagem abaixo.
    pdf = lambda mean: multivariate_normal.pdf(X, mean=mean, cov=self.__sigma)

    # Apply a function to 1-D slices along the given axis
    # No caso aplica a função fdp no axis 1 das médias
    y_probs = np.apply_along_axis(pdf, 1, self.__media) * self.__probabilidadeClasses

    return self.__classes[np.argmax(y_probs, axis=0)]


## Treino

In [19]:
agd = GaussianDiscriminant()
fit = agd.fitAGD(x_train.to_numpy(), y_train)


## Predição

In [20]:
y_predict = agd.predict(x_test.to_numpy())

## Métricas

In [21]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.8158844765342961

Confusion Matrix:
[[105  25]
 [ 26 121]]


In [22]:
report

{'0': {'f1-score': 0.8045977011494252,
  'precision': 0.8015267175572519,
  'recall': 0.8076923076923077,
  'support': 130},
 '1': {'f1-score': 0.825938566552901,
  'precision': 0.8287671232876712,
  'recall': 0.8231292517006803,
  'support': 147},
 'accuracy': 0.8158844765342961,
 'macro avg': {'f1-score': 0.8152681338511631,
  'precision': 0.8151469204224615,
  'recall': 0.815410779696494,
  'support': 277},
 'weighted avg': {'f1-score': 0.8159229979519917,
  'precision': 0.8159828173492073,
  'recall': 0.8158844765342961,
  'support': 277}}

## Salvando resultados

In [23]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd_100dim.json')

In [24]:
resultados

,precision,recall,f1score,y_predict,y_real
0,0.815983,0.815884,0.815923,"[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


In [25]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd_100dim.json')

,precision,recall,f1score,y_predict,y_real
0,0.815983,0.815884,0.815923,"[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


# 5. Árvore de Decisão

## Treino

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

max_depths = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
params = {'criterion': ['gini','entropy'], 'max_depth': max_depths}

clf = GridSearchCV(DecisionTreeClassifier(), params, verbose=2)
    
print("[Árvore de Decisão] Treinando modelos...")
clf.fit(x_train, y_train['classification'].values)

[Árvore de Decisão] Treinando modelos...
Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=5 ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:   11.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                

In [27]:
params = clf.best_params_
print("Parâmetros escolhidos para Árvore de Decisão: ", clf.best_params_)

Parâmetros escolhidos para Árvore de Decisão:  {'criterion': 'gini', 'max_depth': 50}


## Predição

In [28]:
y_predict = clf.predict(x_test)

## Métricas

In [29]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.7184115523465704

Confusion Matrix:
[[ 90  40]
 [ 38 109]]


{'0': {'f1-score': 0.6976744186046512,
  'precision': 0.703125,
  'recall': 0.6923076923076923,
  'support': 130},
 '1': {'f1-score': 0.7364864864864866,
  'precision': 0.7315436241610739,
  'recall': 0.7414965986394558,
  'support': 147},
 'accuracy': 0.7184115523465704,
 'macro avg': {'f1-score': 0.717080452545569,
  'precision': 0.7173343120805369,
  'recall': 0.716902145473574,
  'support': 277},
 'weighted avg': {'f1-score': 0.7182714365780439,
  'precision': 0.7182063637244688,
  'recall': 0.7184115523465704,
  'support': 277}}

## Salvando resultados

In [30]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad_100dim.json')

In [31]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad_100dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.718206,0.718412,0.718271,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'gini', 'max_depth': 50}"


# 6. Random Forest

## Treino

In [32]:
from sklearn.ensemble import RandomForestClassifier

print("[RANDOM FOREST] Selecionando hiperparâmetros...")
num_estimators = [100, 110, 120, 130, 140, 150]
max_depths = [1, 5, 10, 15, 20, 25, 55, 60]

params = {'criterion': ['gini','entropy'],
          'n_estimators': num_estimators,
          'max_depth': max_depths
          }

# params
clt = GridSearchCV(RandomForestClassifier(), params, verbose=2)

print("[RANDOM FOREST] Treinando modelo...")
clf.fit(x_train, y_train['classification'])

[RANDOM FOREST] Selecionando hiperparâmetros...
[RANDOM FOREST] Treinando modelo...
Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[C

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:   11.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                

In [33]:
print("[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest: ", clf.best_params_)

[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest:  {'criterion': 'gini', 'max_depth': 45}


## Predição

In [34]:
y_predict = clf.predict(x_test)

## Métricas

In [35]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.6787003610108303

Confusion Matrix:
[[ 87  43]
 [ 46 101]]


{'0': {'f1-score': 0.6615969581749049,
  'precision': 0.6541353383458647,
  'recall': 0.6692307692307692,
  'support': 130},
 '1': {'f1-score': 0.6941580756013744,
  'precision': 0.7013888888888888,
  'recall': 0.6870748299319728,
  'support': 147},
 'accuracy': 0.6787003610108303,
 'macro avg': {'f1-score': 0.6778775168881397,
  'precision': 0.6777621136173768,
  'recall': 0.678152799581371,
  'support': 277},
 'weighted avg': {'f1-score': 0.6788766847514068,
  'precision': 0.6792121323163504,
  'recall': 0.6787003610108303,
  'support': 277}}

## Salvando métricas

In [36]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest_100dim.json')

In [37]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest_100dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.679212,0.6787,0.678877,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'gini', 'max_depth': 45}"
